# Project 4 - Circuit 4C: DIY Who Am I? Game

Want to create a DIY environmental monitor or weather station? You can use a small, low-cost sensor like the TMP36 to make devices that track and respond to temperature. In this activity you will also use the OLED screen to display sensor readings, a common use for OLEDs in electronics projects.
 
TODO: Update the LCD display with an OLED display.
TODO: Update the jumper wires with a Qwiic connector.
TODO: Remove potentiometer
![What you need](images/sik-demo-prj4-cb-need.png)

## New Components

### TMP36 Temperature Sensor

This temperature sensor has three legs. One connects to 5V, one to ground, and the voltage output from the third leg varies proportionally to changes in temperature. By doing some simple math with this voltage we can measure temperature in degrees Celsius or Fahrenheit.

![TempSensor](images/sik-docs-prj4-cb-temp-sensor.jpg)

## New Concepts

### Algorithms
An algorithm is a process used in order to achieve a desired result. Often, the information needed to create an algorithm lives in the part's datasheet. This sketch uses a few formulas to turn a voltage value into a temperature value, making them all part of the larger temperature-retrieving algorithm. The first formula takes the voltage read on analog pin 0 and multiplies it to get a voltage value from 0V--5V:

```python
voltage = tempSensor.read_u16() * (3.3 / 65536.0)
```
We convert the analog "count" from the temperature sensor that gets returned by `read_u16()` to a voltage by multiplying by the maximum possible voltage (3.3 V) and dividing by the number of different possible counts (since read_u16() returns a 16-bit value that is 2^16 = 65536)

The next formula takes that 0--5V value and calculateds degrees Centigrade:

```python
degreesC = (voltage - 0.5) * 100.0
```
The reason 0.5V is subtracted from the calculated voltage is because there is a 0.5V offset, mentioned on page 8 of the [TMP36 datasheet](https://cdn.sparkfun.com/datasheets/Sensors/Temp/TMP35_36_37.pdf). It's then multiplied by 100 to get a value that matches temperature.

The last formula takes the Centigrade temperature and converts it to a Fahrenheit temperature using the standard conversion formula:

```python
degreesF = degreesC * (9.0 / 5.0) + 32.0
```

Together, these three formulas make up the algorithm that converts from the analog input counts to degrees Fahrenheit.

## Hardware Hookup

The temperature sensor is polarized and can only be inserted in one direction. See below for the pin outs of the temperature sensor. Pay very close attention to the markings on each side as you insert it into your circuit.

![TempPinout](images/sik-docs-prj4-cb-temp-pinout.png)

<span style="color:#a94442">Heads up! Double check the polarity of the TMP36 temperature sensor before powering the RedBoard. It can become very hot if it is inserted backward!</span>

Ready to start hooking everything up? Check out the circuit diagram and hookup table below to see how everything is connected.

### Circuit Diagram

TODO: Replace with new fritzing showing the qwiic conection between the OLED and the RedBoard RP2350
![Hookup](images/sik-docs-prj4-cb-hookup.jpg)

**Note for Advanced Users**: If you know how to read datasheets and schematics, you can also refer to the schematic below as an alternative.

TODO: Replace with new schematic showing the qwiic conection between the OLED and the RedBoard RP2350
![Schematic](images/sik-docs-prj4-cb-schem.jpg)

### Hookup Table

TODO: Change to match the new simple connection of:
 5V -> temp +V 
 GND -> temp GND
 A0 -> temp SIGNAL
 OLED Qwiic -> RedBoard Qwiic

| Component                  | RedBoard           | Breadboard      | Breadboard      | Breadboard      |
|----------------------------|--------------------|-----------------|-----------------|-----------------|
| Jumper Wire                | 5V                 | 5V Rail ( + )   |                 |                 |
| Jumper Wire                | GND                | GND Rail ( - )  |                 |                 |
| LCD                        |                    | A15-A30 (Pin 1 on A15) |          |                 |
| Jumper Wire                |                    | E30             | GND Rail ( - )  |                 |
| Jumper Wire                |                    | E29             | 5V Rail ( + )   |                 |
| Jumper Wire                | Digital Pin 8      | E28             |                 |                 |
| Jumper Wire                | Digital Pin 9      | E27             |                 |                 |
| Jumper Wire                | Digital Pin 10     | E26             |                 |                 |
| Jumper Wire                | Digital Pin 11     | E25             |                 |                 |
| Jumper Wire                | Digital Pin 12     | E20             |                 |                 |
| Jumper Wire                |                    | E19             | GND Rail ( - )  |                 |
| Jumper Wire                | Digital Pin 13     | E18             |                 |                 |
| Jumper Wire                |                    | E16             | 5V Rail ( + )   |                 |
| Jumper Wire                |                    | E15             | GND Rail ( - )  |                 |
| Potentiometer              |                    | A8              | A9              | A10             |
| Jumper Wire                |                    | E9              | E17             |                 |
| Jumper Wire                |                    | E8              | GND Rail ( - )  |                 |
| Jumper Wire                |                    | E10             | 5V Rail ( + )   |                 |
| TMP36 Temperature Sensor   |                    | A1 (GND)        | A2 (Signal)     | A3 (V+)         |
| Jumper Wire                |                    | E1              | GND Rail ( - )  |                 |
| Jumper Wire                | Analog Pin 0 (A0)  | E2              |                 |                 |
| Jumper Wire                |                    | E3              | 5V Rail ( + )   |                 |



## Using the OLED.

Now that your circuit is built, it's time to write to the OLED. This is done using MicroPython, which is running on the RedBoard.

The first step is to connect your RedBoard to a USB port on this computer.

Select the "Connect" button at the bottom right of this screen and a panel is displayed

Select the "Connect Device" Button, and when the selection dialog appears, select the port with that displays ***Board in FS mode (...)*** or *TBD*

![Select a Port](images/sik-demo-select-port.png)

With the RedBoard connected, use the following MicroPython commands to write to the OLED. 

### Using MicroPython

The following MicroPython commands are entered to write to the OLED. 

**REMEMBER** To enter a MicroPython command, hold down either the Control (on Windows) or Command (on Mac) key when pressing *Enter*

#### Step 1 - Setup

Lets start by importing any of the libaries we plan on using and setting up our pins.

In [ ]:
# The qwiic_oled driver module allows for control of SparkFun OLEDs.
# The QwiicLargeOled class is for the Large 1.3" OLED display in our SIK
from qwiic_oled import QwiicLargeOled
from machine import ADC # Allows us to use "ADC" (analog-to-digital conversion) to read from our analog pin
from machine import Pin # Allows us to use "Pin" to use code to interface with the pins on our board

# Define the OLED object that we will use
# Note how we don't have to provide the pins, the driver automatically selects the pins for the qwiic connector
myOLED = QwiicLargeOled()

# Create an ADC object for reading the temperature sensor
tempSensor = ADC(Pin.board.A0)  # A0 is the pin we connected the temperature sensor to


#### Step 2 - Reading Temperature
Now let's write functions to read the temperature in both celcius and Fahrenheit using the "algorithm" that we discussed above. 

In [ ]:
def temperatureC():
    voltage = tempSensor.read_u16() * 3.3 / 65535.0 # Convert the raw reading to a voltage (0-5V)
    degreesC = (voltage - 0.5) * 100.0 # Convert the voltage to degrees Celsius
    return degreesC

def temperatureF():
    degreesC = temperatureC() # Get the temperature in Celsius
    degreesF = (degreesC * 9.0 / 5.0) + 32.0 # Convert Celsius to Fahrenheit
    return degreesF

#### Step 3 - Displaying Temperature on the OLED

Finally, lets read the temperature and display it on the OLED.

In [ ]:
from time import sleep
myOLED.begin() # Initialize the OLED

# Infinite loop to continuously read the temperature and display it
while True:
    tempC = temperatureC() # Read the temperature in Celsius
    tempF = temperatureF() # Read the temperature in Fahrenheit

    # Clear the OLED display
    myOLED.clear(myOLED.ALL) # Clear OLED graphic memory.
    myOLED.clear(myOLED.PAGE) # Clear the processor's display buffer.

    # Print the temperature readings to the OLED
    # Print temperature in Celsius
    myOLED.set_cursor(0, 0) # Set cursor to the top-left corner
    myOLED.print(f"Temp C: {tempC:5}") # Print temperature in Celsius (formatted to 5 characters wide)
    myOLED.set_cursor(0, 10) # Move cursor to the next line (down 10 pixels)
    myOLED.print(f"Temp F: {tempF:5}") # Print temperature in Fahrenheit (formatted to 5 characters wide)

    myOLED.display() # Display the updated text on the OLED

    sleep(1) # Wait for 1 second before reading again

## What You Should See
The OLED will show the temperature in Celsius and Fahrenheit. The temperature readings will update every second. An easy way to see the temperature change is to press your finger to the sensor or to breath on it.

## You've Completed Circuit 4B!

Continue to circuit 4C to build a fun game incorporating your OLED.

![Next - Circuit B](images/sik-demo-prj4-cb-next.png)